In [ ]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from modules.data.video_file import VideoFile
from modules.utils.config_loader import get_config
conf, confp = get_config()

vid_dir = confp.dirs.videos

In [ ]:
def show_img(v, time):
    fig, ax = plt.subplots(figsize=(16, 9))
    ax.axis('off')
    plt.imshow(v.get_frame(time))

In [ ]:
filename = # insert file path
v = VideoFile(path=os.path.join(vid_dir, filename))

In [ ]:
plt.imshow(v.get_frame(10))

In [ ]:
frame_stats = v.get_frame_stats()
frame_stats = [i.split(',') for i in frame_stats.split('\n')]
frame_stats_df = pd.DataFrame(frame_stats[1:], columns=frame_stats[0])
frame_stats_df = frame_stats_df[:-1]
frame_stats_df = frame_stats_df.apply(pd.to_numeric, errors='ignore')
frame_stats_df.rename(columns = {'best_effort_timestamp_time' : 'time'}, inplace=True)
frame_stats_df['time_btw_frames'] = frame_stats_df.time.diff()

In [ ]:
counts = {}
gaps = []
good_frames = []
gap_time = 0
vid_time = 0
last_problem = 0
for i, frame in frame_stats_df.iterrows():
    if frame.time_btw_frames > .201 or frame.time_btw_frames < 0.10:
        n_good_frames = i - last_problem
        good_frames.append(n_good_frames)
        vid_time += (i - last_problem)/25.
        last_problem = i
        try:
            counts[n_good_frames] += 1
        except KeyError:
            counts[n_good_frames] = 1
        gaps.append(frame.time_btw_frames)
        gap_time += frame.time_btw_frames

In [ ]:
plt.bar(list(counts.keys()), counts.values())
plt.show()

In [ ]:
counts

In [ ]:
frames_since_last_i = []
time_since_last_i = []
last_i_frame = []
last_i_time = []
for i, frame in frame_stats_df.iterrows():
    if frame.key_frame:
        last_i_frame.append(i)
        last_i_time.append(frame.time)
        frames_since_last_i.append(0)
        time_since_last_i.append(0)
    else:
        last_i_frame.append(last_i_frame[-1])
        last_i_time.append(last_i_time[-1])
        frames_since_last_i.append(i-last_i_frame[-1])
        time_since_last_i.append(frame.time - last_i_time[-1])

In [ ]:
plt.plot(good_frames, gaps, 'k.')
plt.xlabel('Number of consecutive smooth frames')
plt.ylabel('Gap following those frames (s)')
plt.show()

In [ ]:
plt.plot(time_since_last_i, frames_since_last_i,'k.')
plt.xlabel('Time since last I-frame (s)')
plt.ylabel('Number of frames since last I-frame')

In [ ]:
frame_stats_df['frames_since_last_i'] = frames_since_last_i
frame_stats_df['time_since_last_i'] = time_since_last_i
frame_stats_df['last_i_frame'] = last_i_frame
frame_stats_df['last_i_time'] = last_i_time

In [ ]:
def break_video_into_clips(filename, frame_stats, fps=6, monitor='both'):
    start_time = 0.
    clip_order = 0
    frames_in_clip = 0
    for i, frame in frame_stats.iterrows():
        if not i:
            frames_in_clip += 1
            continue
        if not frames_in_clip:
            if frame.key_frame:
                start_time = frame.time
                frames_in_clip = 1
            continue
        if frame.time_btw_frames < .201 and frame.time_btw_frames > .05:
            frames_in_clip += 1
            end_time = frame.time
        else:
            if frames_in_clip > 10:
                if monitor == 'both':
                    fig, ax = plt.subplots(figsize=(16, 9))
                    ax.axis('off')
                    plt.imshow(v.get_frame(start_time))
                    plt.show()
                    fig, ax = plt.subplots(figsize=(16, 9))
                    ax.axis('off')
                    plt.imshow(v.get_frame(end_time))
                    plt.show()
                elif monitor == 'start':
                    fig, ax = plt.subplots(figsize=(16, 9))
                    ax.axis('off')
                    plt.imshow(v.get_frame(start_time))
                    plt.show()
                elif monitor == 'end':
                    fig, ax = plt.subplots(figsize=(16, 9))
                    ax.axis('off')
                    plt.imshow(v.get_frame(end_time))
                    plt.show()
                print('Saving video clip %i, between times %.3f and %.3f'%(clip_order,start_time,end_time))
                my_clip = v.video_file_clip.subclip(start_time, end_time)
                my_clip.write_videofile('clips/'+filename.split('.')[0]+'_'+str(int(start_time)+1).zfill(4)+'.mkv',
                                        fps=fps,codec='h264')
                clip_order += 1
            if frame.key_frame:
                start_time = frame.time
                frames_in_clip = 1
            else:
                frames_in_clip = 0
    if frames_in_clip > 10:
                print('Saving video clip %i, between times %.3f and %.3f'%(clip_order,start_time,end_time))
                my_clip = v.video_file_clip.subclip(start_time, end_time)
                my_clip.write_videofile('clips/'+filename.split('.')[0]+'_'+str(int(start_time)+1).zfill(4)+'.mkv',
                                        fps=fps,codec='h264')